# Моделируем кубернетес кластер

> Этот ноутбук и соответствующую модель можно пощупать в [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/eosfor/scripting-notes/HEAD)

## Задача

Довольно часто, при проектировании инфраструктуры под Kubernetes (AKS) в Microsoft Azure, у заказчиков возникают странные вопросы: какого размера будет все это добро, или, сколько все это будет нам стоить? Для того чтобы хоть как-то ответить на этот вопрос надо предположить какого размера виртуальные машины выбрать, сколько их должно быть, сколько они стоят в нужном регионе. Размер машин, в свою очередь, зависит от количества и прожорливости приложений, а также от количества экземпляров каждого из них. И так далее. А ведь нам хочется еще и подобрать все это по минимальной цене. В общем и целом, входных переменных довольно много и свести их в кучу, да еще и для каждого размера виртуальных машин отдельно, скажем прямо - не самая интересная работа в мире.

В этом эксперименте мы попробуем построить модель, которая сможет оценить необходимый размер кластера, исходя из количества и размеров приложений. А также будет стараться уменьшить его стоимость.

## Формальные требования

Начнем с виртуальных машин. Первое и, наверное, самое важное - ресурсов машины, должно хватить на все запущенные экземпляры приложения. Другими словами, в сумме, приложения должны сожрать меньше, чем доступно на ноде. Кроме того, здесь стоит учесть нагрузку, которую создает сам Kubernetes. Для этого имеет смысл зарезервировать некий процент ресурсов, и не отдавать его приложениям. С другой стороны, чтобы ресурсы не простаивали, нужно иметь некую нижнюю планку потребления ресурсов. Так же, было бы хорошо, чтобы суммарно, все приложения влезали в кластер, а для этого модель должна определять необходимое количество нод, держа в уме нагрузку на приложения.
Переходим к приложениям. Положим, каждое приложение должно исполняться как минимум в двух экземплярах, на случай отказа одного из них. При этом эти экземпляры должны исполняться на разных нодах, по той же причине. Плюс к этому, было бы очень неплохо, если бы модель умела оценить количество нод, в зависимости от предполагаемой нагрузки на приложение.

## Построение модели

В качестве инструмента для моделирования будем использовать minizinc.

### Моделирование приложений

Начнем с описания приложений. Мы хотим запускать в кластере некое, заранее известное количество приложений, с известными максимальными значениями потребления ресурсов. Все эти значения - переменные, что дает нам возможность настраивать модель и менять ее поведение. Для моделирования "сложных" объектов в minizinc применяется следующий трюк. Задается некий ENUM тип, который используется как индекс в массивах со значениями атрибутов

```powershell
enum appNames = {A, B, C, D, E};
array[appNames] of int: appRAM = [2,4,6,8,4];
array[appNames] of int: appCPU = [200,400,400,450,2250];
array[appNames] of int: appConnectionsPerInstance = [5,2,2, 2,2];
array[appNames] of int: appConnectionsFact = [20,4,3,4,10];
```

Здесь `{A, B, C, D, E}` - имена приложений. Массивы `appRAM`, `appCPU`, `appConnectionsPerInstance`, `appConnectionsFact` объемы потребляемыых ресурсов: памяти, процессора, оценочое количество "подключений", которое может обслужить один экземпляр приложения и фактическое количество таких "подключений"

### Ноды кластера

Нам нужно смоделировать объект ноды с его атрибутами, поэтому

```powershell
enum nodeKinds = {D2sv5, D4sv5, D8sv5}; 
array[nodeKinds] of int: clusterNodeRAM = [8,16,32];
array[nodeKinds] of int: clusterNodeCPU = [2000,4000,8000];
array[nodeKinds] of float: clusterNodePrice = [0.048, 0.096, 0.192];
```

Тут я использую имена размеров виртуальных машин Microsoft Azure, их стоимость и параметры. Но в целом, все ровно то же самое, что и с приложениями

### Decision variables

Стоит сделать отступление и упомянуть, что в minizinc есть два типа "переменных": обычные переменные, к которым мы все привыкли, и "decision variables". Последние представляют "решения", которые модель на minizinc должна принять, чтобы требования и ограничения модели были соблюдены. Иными словами, значения обычных переменных мы должны задать вручную, а значения "decision variables" minizinc должен найти сам. В этом, собственно, вся штука. Мы задаем ограничения, и просим модель подобрать такие значения для "decision variables", которые удовлетворяли бы всем требованиям

В нашем случае подбиать подходящее количество нод в кластере мы будем руками, просто перезапуская модель. Поэтому просто зададим некий изначальный размер, который будет случить верхней границей размера.

```powershell
int: maxClusterSize = 6;
set of int: CLUSTER = 1..maxClusterSize;
```

Теперь начинается самое интересное. На каждой ноде этого кластера мы хотим запусить одно или несколько приложений. Смоделируем это как некую матрицу, в которой измерениями будут ноды кластера и имена приложений. Другими словами мы построим матрицу А размера m x n, где строки m - означают ноды кластера, а столбцы n - приложения. Элемент матрицы a[i,j] - булевская "decision variable", которая показывает запущено ли на ноде i приложение j.

```powershell
array[CLUSTER, appNames] of var bool: runningApps;
```

В этом примере индексами массива служат списки `CLUSTER` и `appNames` а значениями - decision variables.

### Добавляем ограничения

Мы хотим чтобы модель учитывала следующие основные требования:

Суммарное потребление RAM всеми приложениями, исполняющимися на одной ноде должно лежать между некими минимальным и максимальным значениями. Максимальное значение задает ограничение сверху, например мы хотим зарезервировать некий процент памяти для самого кубернетиса. Нижняя граница в свою очередь требует, чтобы нода не была пустой.

```
% sum of RAM of all apps on the node >=minNodeRAMConsumption and <= maxNodeRAMConsumption of total node RAM
constraint forall (i in CLUSTER) ( 
  nodeRAMConsumption[i] >= (clusterNodeRAM[nodes[i]] * minNodeRAMConsumption) /\
  nodeRAMConsumption[i] <= (clusterNodeRAM[nodes[i]] * maxNodeRAMConsumption)
);
```

Ровно то же самое для CPU. 

```
% sum of CPU of all apps on the node >= minNodeCPUConsumption and <= maxNodeCPUConsumption of total node CPU
constraint forall (i in CLUSTER) ( 
  nodeCPUConsumption[i] >= (clusterNodeCPU[nodes[i]] * minNodeCPUConsumption) /\
  nodeCPUConsumption[i] <= (clusterNodeCPU[nodes[i]] * maxNodeCPUConsumption)
);
```

Остался сущий пустяк, мы хотим, чтобы модель учитывала нагрузку на приложения. У нас есть максимальная нагрука на экземпляр - `appConnectionsPerInstance`, после которой необходимо добавить еще один, и текущая фактическая нагрузка - `appConnectionsFact`. На основе этих значений мы можем потребовать, чтобы модель посчитала количество экземпляров каждого приложения. И затем потребовать, чтобы суммарное количество экземпляров приложения в кластере совпадало с рассчетным.

```
% assuming the load, calculate target number of pods
array[appNames] of var int: appInstanceCount;
constraint forall (j in appNames)(
  appInstanceCount[j] = ceil(appConnectionsFact[j]/appConnectionsPerInstance[j])
);

% assuming the load, use calculated number of pods
constraint forall (j in appNames) ( 
  sum(i in CLUSTER) (runningApps[i,j]) = appInstanceCount[j]
);
```

### Немного оптимизации

Кроме всего прочего мы хотим, чтобы цена всего кластера была минимальной. Для этого мы задаем сумму как цель для оптимизации и просим модель минимизировать эту функцию


```
var float: cost =  (sum(i in nodes)(clusterNodePrice[i])) * 24 * 30 ; % target function to optimize
solve minimize cost;
```

## Пощупаем модель

Готовая модель хранится [вот тут](./kubernetes-model/model.mzn).

Запустим ее. Для этого у вас должен быть установлен minizinc. Либо можно попробовать этот ноутбук в [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/eosfor/scripting-notes/HEAD). Контейнер с этим ноутбуком уже содержит minizinc.

In [ ]:
minizinc ./kubernetes-model/model.mzn

D4sv5; CPU:2700/4000; RAM:12/16, D;E;
D4sv5; CPU:2850/4000; RAM:10/16, A;B;E;
D4sv5; CPU:2850/4000; RAM:12/16, A;C;E;
D4sv5; CPU:2850/4000; RAM:12/16, A;C;E;
D4sv5; CPU:2700/4000; RAM:12/16, D;E;
D4sv5; CPU:1000/4000; RAM:12/16, A;B;C;
----------


Как это читать.

```
D4sv5; CPU:2850/4000; RAM:12/16, A;C;E;
```

- `D4sv5` размер виртуальной машины, которую выбрала модель
- `CPU:2850/4000` текущая загрузка/максимальная загрузка ноды
- `RAM:12/16` текущая загрузка/максимальная загрузка ноды
- `A;C;E` приложения, которые модель расположила на этой ноде

Кроме этого есть два специальных индикатора

- `----------` так обозначаются решения, предлагаемые моделью. Их может быть больше одного
- `==========` так отмечается оптимальное решение

В нашем случае модель нашла оптимальное решение и показала его